In [ ]:
pip install jieba_fast

In [2]:
#导入相关包、库
import timeit
import jieba_fast
import os
import math
import csv

#路径设置  
path='C:\\Users\\JYZ'
os.chdir(path)

In [ ]:
# 在路径下 建立dict.txt空文档

In [3]:
%%writefile dict.txt 
狗

Overwriting dict.txt


In [4]:
#检查是否导入
with open('./dict.txt',encoding='utf-8') as f:
    content = f.read()
print(content)

狗



In [5]:
#类
class MySearcherV0:
    def __init__(self):
        self.docs = []
        self.load_data()
        self.cache={}
        self.df = {} #单词和他对应的出现频率
        self.avgdl = 0 #平均文本长度
        self.build_cache()
        jieba.load_userdict("dict.txt")
    def load_data(self):#加载数据
        with open('./final_homework.csv','r') as f:
            reader = csv.reader(f)
            for i in reader:
                for j in range(len(i)):
                    self.docs.append(i[j])
    def build_cache(self):#建立缓存 每个关键词在doc_id内出现
        word_set = set()
        doc_id = 0
        doc_length_sum = 0
        for doc in self.docs:
            doc_word_set = set()
            doc_length_sum += len(doc)  
            for word in jieba_fast.cut(doc,cut_all = True):
                if word not in doc_word_set:
                    result_item = doc_id
                    if word not in self.cache:
                        self.cache[word] = set([result_item])
                    else:
                        self.cache[word].add(result_item)
            #文档频率的计算代码 单词在几个文档中出现
                    if word in self.df:
                        self.df[word] += 1
                    else:
                        self.df[word] = 1
                    doc_word_set.add(word)
            doc_id += 1
        self.avgdl = doc_length_sum/len(self.docs) #---added
        
    def score(self,doc,query,k1 = 2,b=0.75):#BM25
        score = 0
        sum_N = len(self.docs)
        for keyword in jieba.cut(query):
            tf_word_set = set()
            for word in jieba.cut(doc):
                if keyword == word:
                    dl = len(doc)
                    f = doc.count(word)
                    tf = f*(k1+1)/(f+k1*(1-b+b*(dl/self.avgdl)))
                    idf = math.log10((sum_N-self.df[word]+0.5)/(self.df[word]+0.5))
                    score += tf*idf
                tf_word_set.add(word)
        return score
    
    def get_phrase_match(self,phrase):#search 返回含有关键词的doc_id
        result = set([])
        wid = 0
        for word in jieba_fast.cut(phrase):
            if word in self.cache:
                if wid == 0:
                    result = self.cache[word]
                    wid += 1
                else:
                    result = result& self.cache[word]
            else:
                result = result & set([])
        return result
    
    def conv_query(self,query):#将含与或非逻辑关系的查询 进行切分
        query += " "
        qlen = len(query)
        result_parts = []
        idx = 0
        cache =''
        query_keyword = []
        result_id_keyword=[]
        while idx < qlen:
            if query[idx] in ('(',")"," "):
                if cache != "":
                    if cache.lower() =="and":  #lower() replace()
                        result_parts.append("&")
                    elif cache.lower() =="or":
                        result_parts.append("|")
                    elif cache.lower() =="not":
                        result_parts.append("-")
                    else:
                        query_keyword.append(cache)
                        result_parts.append("self.get_phrase_match('{}')".format(cache)) #函数
                    cache =""
                result_parts.append(query[idx])
            else:
                cache += query[idx]
            idx += 1
        result_id_keyword.append(eval(''.join(result_parts)))#doc_id
        result_id_keyword.append(query_keyword)#keyword
        return result_id_keyword


    def search(self,query): #rank
        '''
        新增部分，对结果进行rank排序
        对含有 与或非 逻辑关系的query进行得分排序
        '''
        result = []
        result_final = []
        result_id_keyword = self.conv_query(query)
        for doc_id in result_id_keyword[0]:
            score = 0
            for subquery in result_id_keyword[1]:
                score += self.score(self.docs[doc_id],subquery)
            result.append([doc_id,score])
        result.sort(key = lambda x:x[1],reverse = True)
        result_final.append(result)#doc_id,score
        result_final.append(result_id_keyword[1])#keyword
        return result_final

    def render_search_result_v2(self,query): #格式化输出
        result_all = self.search(query)
        sorted_result_id_list = [x[0] for x in result_all[0]]
        sorted_result_score_list = [x[1] for x in result_all[0]]
        result_keyword_lst = result_all[1]
        print("共检索出",len(sorted_result_id_list),"条数据","\n","--------------分割线-------------")
        for i in range(len(sorted_result_id_list)):
            item = sorted_result_id_list[i]
            score = sorted_result_score_list[i]
            s = self.docs[item]
            for word in result_keyword_lst:
                keyword_list = []
                for keyword in jieba.cut(word):
                    keyword_list.append(keyword)
                for keyword in keyword_list:
                    s = s.replace(keyword,'*{}*'.format(keyword)).strip()
            print("编号：{}    score：{} \n{}".format(item,score,s))
            print("--------------分割线--------------")

In [6]:
%%time
searcher_1x = MySearcherV0()

FileNotFoundError: [Errno 2] No such file or directory: './final_homework.csv'

In [2]:
%%time
searcher_1x.search('慢性炎症')

In [ ]:
searcher_1x.render_search_result_v2('慢性炎症')

In [3]:
%%time
searcher_1x.search('(慢性炎症 or 病变) and 右肺')

In [21]:
searcher_1x.render_search_result_v2('(慢性炎症 or 病变) and 右肺')